# Application de methode de deep learning à un fonds d'images d'archives

## Du pixel aux images - 32M7138

*Printemps 2025 - Université de Genève*

*Raphaël Rollinet (raphael.rollinet@unine.ch)*

## Introduction

Ce Markdown effectué dans le cadre du cours "Du pixel aux images : introduction au traitement des images 2D" traite de méthode de deep learning appliqué au traitement de l'image, plus précisément de la détection d'objet sur un fonds d'image ancienne. 

### Méthode de détection d'objet

La méthode utilisée dans le cadre de ce projet est la suivante

### Fonds d'images de la commune de Milvignes

![alt text](Milvignes-blazon.svg)

Le fonds d'images provient de l'inventaire des archives de la commune de Milvignes, membre du service intercommunal d'archives de Neuchâtel, la plateforme de diffusion des inventaires est gérée par docuteam SA, entreprise de gestion d'archives qui m'a employé de 2016 à 2021. Les images numérisées par mes soins en format de conservation (TIFF) et archivées électroniquement dans un repository de préservation numérique et également disponible  disponible en téléchargement librement dans un format dégradé de diffusion (JPEG). 

Les images sont disponibles ici : https://milvignes.docuteam.cloud/fr/units/1-archives-communales-de-milvignes-1326-2013/gallery



## Problématique

Ayant longuement travaillé dans le domaine des archives, la thématique d'appliquer les méthodes vues dans le cadre de ce cours sur un fonds d'archives est un sujet intéressant dans son potentiel d'application sur un fonds d'archives d'images. Dans les institutions patrimoniales, les images sont un support courant, ma problématique sera donc appliquée à un fonds ancien d'images numérisées de la commune de Milvignes (NE). Mon questionnement porte sur l'application de cette méthode utilisant des modèles de deep learning entrainé sur des images relativement récentes, mais ici appliqué à un fonds d'images anciennes (~1900).

### Question de recherche

- "Quel est le potentiel de la détection d'objet dans le traitement de fonds d'images en archives ?"

## Méthodologie

Le code utilisé est adapté du code vu en cours, il vise à effectuer une détection d'objet sur des images, tout en répondant à des contraintes métiers qui sont par exemple un besoin d'automatiser le traitement à partir d'un workflow. Mais également en termes de rendu, un archiviste aura besoin d'extraire les détections sous format CSV afin qu'elles soient intégrées aux notices d'un système d'information archivistique par exemple sous la forme d'une indexation. Le code permet à n'importe quel archiviste avec des notions informatiques minimales de l'exécuter.

In [ ]:
#Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Installation des librairies (si nécessaire)
!pip install opencv-python
!pip install tensorflow_hub
!pip install -q transformers

In [ ]:
# Importation des librairies Python
import os
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import pandas as pd
from PIL import Image
from transformers import ViTImageProcessor, ViTForImageClassification

In [ ]:
# Charger les modèles (détection + classification)

# Modèle de détection SSD MobileNet V2
detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1")

# Modèle ViT pour classification d'image
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
vit_model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

In [ ]:
# Liste des noms de classes (80 classes COCO)
classe = np.array([
    'background', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
    'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack',
    'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
    'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
    'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink',
    'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
    'hair drier', 'toothbrush'
])
classes = np.concatenate([classe, np.repeat('none', 100)])


In [ ]:
#  À adapter : Mettre le dossier Images dans Google Drive
folder_path = "/content/drive/MyDrive/Colab_Notebooks/images"  # Adapter le chemin selon l'utilisateur

In [ ]:
# Liste pour stocker les résultats
results = []

In [ ]:
# Création d'une fonction de detection et classification pour le traitement de l'image
## Traite une image avec SSD MobileNet (détection) et ViT (classification).
## Sauvegarde l'image annotée et stocke les résultats dans la liste globale 'results'.

def process_image_with_detection_and_classification(image_path):

    # Lire l'image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Erreur : impossible de lire {image_path}")
        return
    
    # Convertir en RGB pour affichage et modèles
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    input_tensor = tf.convert_to_tensor(image_rgb)[tf.newaxis, ...]

    # Détection d'objets
    detections = detector(input_tensor)
    boxes = detections['detection_boxes'][0].numpy()
    scores = detections['detection_scores'][0].numpy()
    detected_classes = detections['detection_classes'][0].numpy().astype(np.uint8)

    # Préparer les annotations et objets détectés
    object_labels = []
    height, width, _ = image.shape
    for i in range(len(scores)):
        if scores[i] < 0.3:
            continue
        label = classes[detected_classes[i]]
        object_labels.append(label)

        # Boîte englobante
        box = boxes[i] * np.array([height, width, height, width])
        box = box.astype(np.int32)
        cv2.rectangle(image_rgb, (box[1], box[0]), (box[3], box[2]), (0, 255, 0), 2)
        cv2.putText(image_rgb, label, (box[1], box[0]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Classification avec ViT
    pil_image = Image.fromarray(image_rgb)
    inputs = processor(images=pil_image, return_tensors="pt")
    outputs = vit_model(**inputs)
    logits = outputs.logits
    predicted_class_idx = logits.argmax(-1).item()
    predicted_label = vit_model.config.id2label[predicted_class_idx]

    # Sauvegarde de l’image annotée dans /Resultat/annotated
    result_folder = os.path.join(folder_path, "Resultat")
    annotated_folder = os.path.join(result_folder, "annotated")
    os.makedirs(annotated_folder, exist_ok=True)

    save_path = os.path.join(annotated_folder, os.path.basename(image_path))
    cv2.imwrite(save_path, cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR))

    # Affichage pour visualisation
    plt.figure(figsize=(8, 6))
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.title(f"{os.path.basename(image_path)}\n→ ViT : {predicted_label}")
    plt.show()

    # Stockage des résultats pour export CSV
    results.append({
        "image": os.path.basename(image_path),
        "detected_objects": ", ".join(object_labels),
        "vit_prediction": predicted_label
    })



In [ ]:
# Appliquer à toutes les images présentes dans le dossiers "images"
supported_formats = ('.jpg', '.jpeg', '.png', '.bmp')
image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(supported_formats)]

print(f"{len(image_files)} images trouvées.")

for image_path in image_files:
    process_image_with_detection_and_classification(image_path)



In [ ]:
# Créer un sous-dossier "Résultat" dans le dossier Drive d'origine
result_folder = os.path.join(folder_path, "Resultat")
os.makedirs(result_folder, exist_ok=True)

# Définir le chemin de sortie du CSV dans ce dossier
csv_output_path = os.path.join(result_folder, "resultats_detection_classification.csv")

# Export du DataFrame en CSV
df_results = pd.DataFrame(results)
df_results.to_csv(csv_output_path, index=False, encoding='utf-8')

print(f"CSV enregistré ici : {csv_output_path}")


## Résultat

## Mise en perspectives et conclusion

## Bibliographie